In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
class LoadData():
    # 加载数据,
    def __init__(self, path="./Data/", dataset="frappe", loss_type="square_loss"):
        self.dataset = dataset
        self.loss_type = loss_type
        self.path = path + dataset + "/"
        self.trainfile = self.path + dataset + ".train.libfm"
        self.testfile = self.path + dataset + ".test.libfm"
        self.validationfile = self.path + dataset + ".validation.libfm"
        self.features_M = {}
        self.construct_df()

    #         self.Train_data, self.Validation_data, self.Test_data = self.construct_data( loss_type )

    def construct_df(self):
        self.data_train = pd.read_table(self.trainfile, sep=" ", header=None, engine='python')
        self.data_test = pd.read_table(self.testfile, sep=" ", header=None, engine="python")
        self.data_valid = pd.read_table(self.validationfile, sep=" ", header=None, engine="python")
        #       第一列是标签，y

        for i in self.data_test.columns[1:]:
            self.data_test[i] = self.data_test[i].apply(lambda x: int(x.split(":")[0]))
            self.data_train[i] = self.data_train[i].apply(lambda x: int(x.split(":")[0]))
            self.data_valid[i] = self.data_valid[i].apply(lambda x: int(x.split(":")[0]))

        # self.all_data = pd.concat([self.data_train, self.data_test, self.data_valid])
        
        self.field_dims = []

        for i in self.all_data.columns[1:]:

            maps = {val: k for k, val in enumerate(set(self.all_data[i]))}
            self.data_test[i] = self.data_test[i].map(maps)
            self.data_train[i] = self.data_train[i].map(maps)
            self.data_valid[i] = self.data_valid[i].map(maps)
            self.features_M[i] = maps
            self.field_dims.append(len(set(self.all_data[i])))

In [20]:
features_M = {}
trainfile = "ml-tag.train.libfm"
testfile = "ml-tag.test.libfm"
validationfile = "ml-tag.validation.libfm"     
data_train = pd.read_table(trainfile, sep=" ", header=None, engine='python')
data_test = pd.read_table(testfile, sep=" ", header=None, engine="python")
data_valid = pd.read_table(validationfile, sep=" ", header=None, engine="python")
#       第一列是标签，y

for i in data_test.columns[1:]:
    data_test[i] = data_test[i].apply(lambda x: int(x.split(":")[0]))
    data_train[i] = data_train[i].apply(lambda x: int(x.split(":")[0]))
    data_valid[i] = data_valid[i].apply(lambda x: int(x.split(":")[0]))

all_data = pd.concat([data_train, data_test, data_valid])
field_dims = []

for i in all_data.columns[1:]:
    # if self.dataset != "frappe":
        # maps = {}
    maps = {val: k+offsets[i-1] for k, val in enumerate(set(all_data[i]))}
    data_test[i] = data_test[i].map(maps)
    data_train[i] = data_train[i].map(maps)
    data_valid[i] = data_valid[i].map(maps)
    features_M[i] = maps
    field_dims.append(len(set(all_data[i])))

In [21]:
data_train.describe()

,0,1,2,3
count,1.605487e+06,1.605487e+06,1.605487e+06,1.605487e+06
mean,-3.338744e-01,8.629221e+03,2.043862e+04,6.083390e+04
std,9.426179e-01,5.029680e+03,4.368120e+03,1.505964e+04
min,-1.000000e+00,0.000000e+00,1.704500e+04,4.078800e+04
25%,-1.000000e+00,4.098000e+03,1.764900e+04,4.703700e+04
50%,-1.000000e+00,9.157000e+03,1.881800e+04,5.861900e+04
75%,1.000000e+00,1.284000e+04,2.118300e+04,7.364000e+04
max,1.000000e+00,1.704400e+04,4.078700e+04,9.044400e+04


In [19]:
offsets = np.array([0,*np.cumsum(field_dims)[:-1]])
print(offsets)

[    0 17045 40788]


In [23]:
data_train.to_csv("ml-tag2.train.libfm", sep=" ",header=None,index=None)
data_valid.to_csv("ml-tag2.valid.libfm",sep=" ",header=None,index=None)
data_test.to_csv("ml-tag2.test.libfm",sep=" ",header=None,index=None)

In [26]:
import numpy as np
import pandas as pd
import torch
import os
import tqdm
import pickle



class LoadData():
    # 加载数据,
    def __init__(self, path="./Data/", dataset="frappe", loss_type="square_loss"):
        self.dataset = dataset
        # self.loss_type = loss_type
        self.path = path + dataset + "/"
        self.trainfile = self.path + dataset + ".train.libfm"
        self.testfile = self.path + dataset + ".test.libfm"
        self.validationfile = self.path + dataset + ".validation.libfm"
        # self.features_M = {}
        self.construct_df()

    #         self.Train_data, self.Validation_data, self.Test_data = self.construct_data( loss_type )

    def construct_df(self):
        self.data_train = pd.read_table(self.trainfile, sep=" ", header=None, engine='python')
        self.data_test = pd.read_table(self.testfile, sep=" ", header=None, engine="python")
        self.data_valid = pd.read_table(self.validationfile, sep=" ", header=None, engine="python")
        #       第一列是标签，y

        # for i in self.data_test.columns[1:]:
        #     self.data_test[i] = self.data_test[i].apply(lambda x: int(x.split(":")[0]))
        #     self.data_train[i] = self.data_train[i].apply(lambda x: int(x.split(":")[0]))
        #     self.data_valid[i] = self.data_valid[i].apply(lambda x: int(x.split(":")[0]))

        # self.all_data = pd.concat([self.data_train, self.data_test, self.data_valid])
        # self.field_dims = []

        # for i in self.all_data.columns[1:]:
        #
        #     maps = {val: k for k, val in enumerate(set(self.all_data[i]))}
        #     self.data_test[i] = self.data_test[i].map(maps)
        #     self.data_train[i] = self.data_train[i].map(maps)
        #     self.data_valid[i] = self.data_valid[i].map(maps)
        #     self.features_M[i] = maps
        #     self.field_dims.append(len(set(self.all_data[i])))


class RecData():
    # define the dataset
    def __init__(self, all_data):
        self.data_df = all_data

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        x = self.data_df.iloc[idx].values[1:]
        y1 = self.data_df.iloc[idx].values[0]
        return x, y1

def getdataloader_frappe(path="../.././data/data2/",dataset="ml-tag2",num_ng=4,batch_size=256):
    # 加载数据
    print("load frappe data")
    DataF = LoadData(path=path, dataset=dataset)
    # 直接保存
    datatrain = RecData(DataF.data_train)
    datavalid = RecData(DataF.data_valid)
    datatest = RecData(DataF.data_test)
    print("datatest",len(datatest))
    print("datatrain",len(datatrain))
    print("datavalid",len(datavalid))
    trainLoader = torch.utils.data.DataLoader(datatrain, batch_size=batch_size, shuffle=True, num_workers=32, pin_memory=True,drop_last=True)
    validLoader = torch.utils.data.DataLoader(datavalid, batch_size=batch_size, shuffle=False, num_workers=4,pin_memory=True)
    testLoader = torch.utils.data.DataLoader(datatest, batch_size=batch_size, shuffle=False, num_workers=4,pin_memory=True)
    return trainLoader, validLoader, testLoader,datatrain

In [27]:
trainLoader, validLoader, testLoader, datatrain =  getdataloader_frappe(path="../")

load frappe data
datatest 200659
datatrain 1605487
datavalid 200640


In [28]:
iters = iter(testLoader)
a  = iters.next()

In [29]:
x,y = datatrain[0]

In [31]:
a


[tensor([[1.6757e+04, 1.7869e+04, 4.1303e+04],
         [8.9790e+03, 1.9052e+04, 8.4916e+04],
         [1.0368e+04, 3.9733e+04, 6.3146e+04],
         [5.1940e+03, 1.7056e+04, 7.3874e+04],
         [9.9290e+03, 1.8724e+04, 4.3090e+04],
         [1.6651e+04, 1.7522e+04, 4.3289e+04],
         [1.0622e+04, 2.0697e+04, 4.8156e+04],
         [1.3156e+04, 1.9183e+04, 4.8518e+04],
         [1.4520e+04, 1.9058e+04, 5.7988e+04],
         [1.0622e+04, 3.1664e+04, 4.6053e+04],
         [1.0622e+04, 1.8242e+04, 5.3608e+04],
         [1.1700e+03, 2.9620e+04, 8.5096e+04],
         [8.8500e+02, 1.7612e+04, 4.9388e+04],
         [9.1570e+03, 3.7559e+04, 4.7192e+04],
         [1.3563e+04, 2.5656e+04, 6.3870e+04],
         [3.6630e+03, 1.7852e+04, 4.2873e+04],
         [7.2590e+03, 3.1863e+04, 8.4022e+04],
         [5.7980e+03, 1.7842e+04, 5.4159e+04],
         [1.1050e+03, 1.7613e+04, 8.0229e+04],
         [1.8980e+03, 1.8784e+04, 5.5719e+04],
         [1.2770e+04, 2.0375e+04, 6.0668e+04],
         [1.5